In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/clean_raw_data.csv
/kaggle/input/new-raw-mooccubex/clean_data_mean.csv
/kaggle/input/new-raw-mooccubex/raw_data.csv
/kaggle/input/new-raw-mooccubex/clean_data_GCN.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/FillZero_m

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}

## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # RNN layer 1
        x = layers.SimpleRNN(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # RNN layer 2
        x = layers.SimpleRNN(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'rnn_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 03m 16s]
val_accuracy: 0.7858450412750244

Best val_accuracy So Far: 0.7895057797431946
Total elapsed time: 00h 31m 46s
Best Parameters for Week 1:
units_1: 192
dropout_1: 0.1
units_2: 96
dropout_2: 0.4
learning_rate: 0.0004055150366826125

Best Parameters for Week 2:
units_1: 160
dropout_1: 0.1
units_2: 64
dropout_2: 0.1
learning_rate: 0.00039212961505715734

Best Parameters for Week 3:
units_1: 96
dropout_1: 0.1
units_2: 192
dropout_2: 0.4
learning_rate: 0.00026046409796031013

Best Parameters for Week 4:
units_1: 256
dropout_1: 0.30000000000000004
units_2: 192
dropout_2: 0.5
learning_rate: 0.0003647863570162414


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_RNN_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # RNN layer 1
    x = layers.SimpleRNN(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # RNN layer 2
    x = layers.SimpleRNN(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
        input_shape = (X_train.shape[1], X_train.shape[2])  # Lấy kích thước từ dữ liệu thực tế
        # Xây dựng mô hình với tham số tốt nhất
        model = build_RNN_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 192, 'dropout_1': 0.1, 'units_2': 96, 'dropout_2': 0.4, 'learning_rate': 0.0004055150366826125}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.5344 - loss: 0.2176 - val_accuracy: 0.6676 - val_loss: 0.1193
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6275 - loss: 0.1469 - val_accuracy: 0.6954 - val_loss: 0.1121
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6475 - loss: 0.1334 - val_accuracy: 0.6851 - val_loss: 0.1097
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6495 - loss: 0.1278 - val_accuracy: 0.6946 - val_loss: 0.1079
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6618 - loss: 0.1234 - val_accuracy: 0.7022 - val_loss: 0.1086
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - accuracy: 0.5447 - loss: 0.2094 - val_accuracy: 0.6878 - val_loss: 0.1174
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6266 - loss: 0.1440 - val_accuracy: 0.6889 - val_loss: 0.1158
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6470 - loss: 0.1344 - val_accuracy: 0.6874 - val_loss: 0.1136
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6468 - loss: 0.1277 - val_accuracy: 0.6870 - val_loss: 0.1118
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6570 - loss: 0.1252 - val_accuracy: 0.6878 - val_loss: 0.1103
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6700 - loss: 0.1207 - val_accuracy: 0.6843 - val_loss: 0.1125
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6785 - loss: 0.1188 - val_accuracy: 0.7042 - val_loss: 0.1058
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6878 - loss: 0.1122 - val_accuracy: 0.6859 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 34ms/step - accuracy: 0.5456 - loss: 0.2037 - val_accuracy: 0.6632 - val_loss: 0.1232
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6312 - loss: 0.1438 - val_accuracy: 0.6793 - val_loss: 0.1213
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6433 - loss: 0.1335 - val_accuracy: 0.6861 - val_loss: 0.1216
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6591 - loss: 0.1300 - val_accuracy: 0.6770 - val_loss: 0.1122
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6723 - loss: 0.1254 - val_accuracy: 0.6732 - val_loss: 0.1128
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6768 - loss: 0.1195 - val_accuracy: 0.6812 - val_loss: 0.1117
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6762 - loss: 0.1193 - val_accuracy: 0.6632 - val_loss: 0.1128
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6825 - loss: 0.1162 - val_accuracy: 0.6930 - va

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.5323 - loss: 0.2279 - val_accuracy: 0.6831 - val_loss: 0.1325
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5924 - loss: 0.1743 - val_accuracy: 0.6834 - val_loss: 0.1352
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6072 - loss: 0.1596 - val_accuracy: 0.6793 - val_loss: 0.1231
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6271 - loss: 0.1427 - val_accuracy: 0.6991 - val_loss: 0.1144
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6475 - loss: 0.1331 - val_accuracy: 0.6831 - val_loss: 0.1114
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6598 - loss: 0.1266 - val_accuracy: 0.6960 - val_loss: 0.1108
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6747 - loss: 0.1255 - val_accuracy: 0.7079 - val_loss: 0.1061
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6728 - loss: 0.1238 - val_accuracy: 0.7132 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 30ms/step - accuracy: 0.5378 - loss: 0.2253 - val_accuracy: 0.6636 - val_loss: 0.1429
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5909 - loss: 0.1739 - val_accuracy: 0.6682 - val_loss: 0.1260
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6143 - loss: 0.1511 - val_accuracy: 0.6720 - val_loss: 0.1216
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6299 - loss: 0.1416 - val_accuracy: 0.6796 - val_loss: 0.1139
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6495 - loss: 0.1331 - val_accuracy: 0.6899 - val_loss: 0.1194
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6641 - loss: 0.1294 - val_accuracy: 0.6930 - val_loss: 0.1154
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6839 - loss: 0.1221 - val_accuracy: 0.6834 - val_loss: 0.1125
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6808 - loss: 0.1194 - val_accuracy: 0.6926 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.7271192789077758
Average Train Time: 25.5923 seconds
Average Test Time: 0.7917 seconds
Average AUC Macro: 0.8565060222008862
Average AUC Weighted: 0.8751403607302397

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.584634        0.567000          0.566583     0.816459
1      1           0.000000        0.000000          0.000000     0.833114
2      2           0.651438        0.446423          0.512267     0.849441
3      3           0.516125        0.378443          0.380870     0.861624
4      4           0.793241        0.871642          0.829896     0.861238

Average Confusion Matrix:
       0    1     2     3       4
0  340.2  0.0  14.4  27.8   217.6
1   37.6  0.0   1.6   8.0    40.4
2   34.6  0.0  73.4   6.6    49.8
3   41.8  0.0   2.8  63.2    59.2
4  140.6  0.0  24.0  41.2  1397.6

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])  # Lấy kích thước từ dữ liệu thực tế
    model = build_RNN_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=20, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.3935 - loss: 0.2501 - val_accuracy: 0.4610 - val_loss: 0.3027
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5127 - loss: 0.1920 - val_accuracy: 0.4398 - val_loss: 0.3087
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5484 - loss: 0.1791 - val_accuracy: 0.4033 - val_loss: 0.3377
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5600 - loss: 0.1727 - val_accuracy: 0.5218 - val_loss: 0.2510
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5724 - loss: 0.1676 - val_accuracy: 0.5313 - val_loss: 0.2452
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5777 - loss: 0.1652 - val_accuracy: 0.5113 - val_loss: 0.2677
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5802 - loss: 0.1650 - val_accuracy: 0.4961 - val_loss: 0.2726
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4365 - loss: 0.2174 - val_accuracy: 0.4717 - val_loss: 0.2281
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5248 - loss: 0.1873 - val_accuracy: 0.4228 - val_loss: 0.3135
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5564 - loss: 0.1731 - val_accuracy: 0.5747 - val_loss: 0.2386
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5731 - loss: 0.1641 - val_accuracy: 0.5171 - val_loss: 0.2665
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5871 - loss: 0.1597 - val_accuracy: 0.4662 - val_loss: 0.2905
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5926 - loss: 0.1578 - val_accuracy: 0.5877 - val_loss: 0.2057
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5971 - loss: 0.1559 - val_accuracy: 0.6323 - val_loss: 0.2204
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.3951 - loss: 0.2421 - val_accuracy: 0.4575 - val_loss: 0.2655
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5031 - loss: 0.1944 - val_accuracy: 0.4263 - val_loss: 0.3025
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5379 - loss: 0.1810 - val_accuracy: 0.4462 - val_loss: 0.3010
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5481 - loss: 0.1748 - val_accuracy: 0.5610 - val_loss: 0.2094
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5609 - loss: 0.1681 - val_accuracy: 0.4388 - val_loss: 0.3296
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5640 - loss: 0.1674 - val_accuracy: 0.5597 - val_loss: 0.2218
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5798 - loss: 0.1622 - val_accuracy: 0.5253 - val_loss: 0.2397
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.3602 - loss: 0.2910 - val_accuracy: 0.4315 - val_loss: 0.2585
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4671 - loss: 0.2078 - val_accuracy: 0.4285 - val_loss: 0.3347
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5103 - loss: 0.1885 - val_accuracy: 0.4335 - val_loss: 0.2764
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5386 - loss: 0.1783 - val_accuracy: 0.5478 - val_loss: 0.2255
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5508 - loss: 0.1739 - val_accuracy: 0.5146 - val_loss: 0.2595
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5545 - loss: 0.1709 - val_accuracy: 0.4315 - val_loss: 0.3216
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5687 - loss: 0.1650 - val_accuracy: 0.5293 - val_loss: 0.2800
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 45.39 seconds
  Test Time: 1.01 seconds
  Accurancy: 0.6725609756097561
  Precision: [0.68727273 0.1461794  0.47826087 0.51219512 0.91041162]
  Recall: [0.504      0.81481481 0.53398058 0.6        0.74975075]
  F1-Score: [0.58153846 0.24788732 0.50458716 0.55263158 0.82230727]
  Macro Precision: 0.5468639486117098
  Macro Recall: 0.6405092290191632
  Macro F1-Score: 0.5417903584251964
  Confusion Matrix:
[[189 116  17   3  50]
 [  3  44   1   0   6]
 [ 17  23  55   1   7]
 [  8  17   6  63  11]
 [ 58 101  36  56 752]]

=== AUC-ROC ===
AUC Macro: 0.8639
AUC Weighted: 0.8677
AUC per Label: [0.84672675 0.87348793 0.82684716 0.89574376 0.87649688]
Week 2:
  Train Time: 45.03 seconds
  Test Time: 0.97 seconds
  Accurancy: 0.7219512195121951
  Precision: [0.64095745 0.17171717 0.64197531 0.65671642 0.88562092]
  Recall: [0.64266667 0.62962963 0.50485437 0.41904762 0.8105683 ]
  F1-Score: [0.64181092 0.26984127 0.56521739 0.51162791